## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import sqlite3

# Read database from 'Data/shill_bidding.sqlite'.

conn = sqlite3.connect('Data/shill_bidding.sqlite')
df = pd.read_sql("select * from bidding",conn)

# Close database
conn.close()

df.head()

,Record_ID,Auction_ID,Bidder_ID,Bidder_Tendency,Bidding_Ratio,Successive_Outbidding,Last_Bidding,Auction_Bids,Starting_Price_Average,Early_Bidding,Winning_Ratio,Auction_Duration,Class
0,1,732,_***i,0.200000,0.400000,0.0,0.000028,0.0,0.993593,0.000028,0.666667,5,0
1,2,732,g***r,0.024390,0.200000,0.0,0.013123,0.0,0.993593,0.013123,0.944444,5,0
2,3,732,t***p,0.142857,0.200000,0.0,0.003042,0.0,0.993593,0.003042,1.000000,5,0
3,4,732,7***n,0.100000,0.200000,0.0,0.097477,0.0,0.993593,0.097477,1.000000,5,0
4,5,900,z***z,0.051282,0.222222,0.0,0.001318,0.0,0.000000,0.001242,0.500000,7,0


In [3]:
# Drop the non-beneficial ID columns, 'Record_ID' and 'Auction_ID' and 'Bidder_ID'.
df=df.drop(columns=['Record_ID','Auction_ID','Bidder_ID'],axis=1)
df.head()

,Bidder_Tendency,Bidding_Ratio,Successive_Outbidding,Last_Bidding,Auction_Bids,Starting_Price_Average,Early_Bidding,Winning_Ratio,Auction_Duration,Class
0,0.200000,0.400000,0.0,0.000028,0.0,0.993593,0.000028,0.666667,5,0
1,0.024390,0.200000,0.0,0.013123,0.0,0.993593,0.013123,0.944444,5,0
2,0.142857,0.200000,0.0,0.003042,0.0,0.993593,0.003042,1.000000,5,0
3,0.100000,0.200000,0.0,0.097477,0.0,0.993593,0.097477,1.000000,5,0
4,0.051282,0.222222,0.0,0.001318,0.0,0.000000,0.001242,0.500000,7,0


In [4]:
# Determine the number of unique values in each column.
df.nunique()

Bidder_Tendency            489
Bidding_Ratio              400
Successive_Outbidding        3
Last_Bidding              5807
Auction_Bids                49
Starting_Price_Average      22
Early_Bidding             5690
Winning_Ratio               72
Auction_Duration             5
Class                        2
dtype: int64

In [6]:
# Split our preprocessed data into our features and target arrays
y = df['Class'].values
X = df.drop('Class', axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [7]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  7
hidden_nodes_layer2 = 14
hidden_nodes_layer3 = 21

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 7)                 70        
                                                                 
 dense_8 (Dense)             (None, 14)                112       
                                                                 
 dense_9 (Dense)             (None, 21)                315       
                                                                 
 dense_10 (Dense)            (None, 1)                 22        
                                                                 
Total params: 519
Trainable params: 519
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [19]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train, epochs=100)

Epoch 1/100
149/149 [==============================] - 4s 3ms/step - loss: 0.3098 - accuracy: 0.8905
Epoch 2/100
149/149 [==============================] - 0s 3ms/step - loss: 0.1355 - accuracy: 0.9270
Epoch 3/100
149/149 [==============================] - 0s 3ms/step - loss: 0.0716 - accuracy: 0.9724
Epoch 4/100
149/149 [==============================] - 0s 3ms/step - loss: 0.0530 - accuracy: 0.9806
Epoch 5/100
149/149 [==============================] - 1s 3ms/step - loss: 0.0472 - accuracy: 0.9819
Epoch 6/100
149/149 [==============================] - 0s 3ms/step - loss: 0.0441 - accuracy: 0.9846
Epoch 7/100
149/149 [==============================] - 0s 2ms/step - loss: 0.0432 - accuracy: 0.9833
Epoch 8/100
149/149 [==============================] - 0s 3ms/step - loss: 0.0416 - accuracy: 0.9838
Epoch 9/100
149/149 [==============================] - 1s 9ms/step - loss: 0.0411 - accuracy: 0.9844
Epoch 10/100
149/149 [==============================] - 1s 7ms/step - loss: 0.0394 - accura

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

50/50 - 1s - loss: 0.0354 - accuracy: 0.9956 - 1s/epoch - 21ms/step
Loss: 0.0354432575404644, Accuracy: 0.9955724477767944


In [12]:
# Export our model to HDF5 file
nn.save("Shill_Bidding.h5")